In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pyarrow.parquet as pq
import sys
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import math
from IPython.display import clear_output
import logging
from pathlib import Path

sys.path.append('../../../')

from configs.data_configs.rosbank import data_configs
from configs.model_configs.supervised.rosbank import model_configs
from src.data_load.dataloader import create_data_loaders, create_test_loader

from src.create_embeddings import create_embeddings

from src.data_load import split_strategy
from src.data_load.data_utils import prepare_data
from src.data_load.splitting_dataset import (
    ConvertingTrxDataset,
    TargetDataset,
    DropoutTrxDataset,
    SplittingDataset,
    TargetEnumeratorDataset,
)
from src.data_load.dataloader import collate_splitted_rows, padded_collate, PaddedBatch
from torch.utils.data import DataLoader

from src.models.preprocessors import FeatureProcessor

from torch.autograd import Variable

from src.models.ckconv.model import CKCONV_Classifier

In [2]:
conf = data_configs()
model_conf = model_configs()

In [3]:
net = CKCONV_Classifier(model_conf=model_conf, data_conf=conf)

/home/viktor/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [4]:
train_loader, valid_loader = create_data_loaders(conf)

for batch in train_loader:
    break

0it [00:00, ?it/s]

9717it [00:00, 11104.63it/s]

Data shapes: train 3915, val 438, test 0


In [5]:
out = net(batch[0])

In [9]:
net.loss(out, batch[1])

{'total_loss': tensor(0.6857, grad_fn=<NllLossBackward0>)}

In [56]:
net = seqImg_CKCNN(in_channels = 5,
        out_channels=4,
        hidden_channels = 13,
        num_blocks = 2,
        kernelnet_hidden_channels = 2,
        kernelnet_activation_function = 'Sine',
        kernelnet_norm_type = '',
        dim_linear = 1,
        bias = True,
        omega_0 = 30,
        dropout = 0.0,
        weight_dropout = 0.0,
        pool = False,  # Always False in our experiments.
        )



/home/viktor/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [58]:
outputs = net(x)

In [59]:
if len(outputs.shape) == 1:
    preds = (outputs > 0.0).int()
else:
    _, preds = torch.max(outputs, 1)

In [60]:
preds

tensor([3, 3, 3])

In [3]:
import ckconv.nn

in_channels = 5
out_channels = 9
kernelnet_hidden_channels = 3
kernelnet_activation_function = 'Sine'
kernelnet_norm_type = ''
dim_linear = 1
bias = False
omega_0 = 30
dropout = 0.0
weight_dropout = 0.0

ckonv = ckconv.nn.CKConv(
            in_channels,
            out_channels,
            kernelnet_hidden_channels,
            kernelnet_activation_function,
            kernelnet_norm_type,
            dim_linear,
            bias,
            omega_0,
            weight_dropout,
        )

/home/viktor/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [7]:
out = ckonv(x)

In [8]:
out.size(), x.size()

(torch.Size([3, 9, 7]), torch.Size([3, 5, 7]))

In [12]:
rel_pos = ckonv.handle_rel_positions(x)
rel_pos

tensor([[[-1.0000e+00, -6.6667e-01, -3.3333e-01, -2.9802e-08,  3.3333e-01,
           6.6667e-01,  1.0000e+00]]])

In [15]:
conv_kernel = ckonv.Kernel(rel_pos).view(-1, x.shape[1], *x.shape[2:])

In [17]:
conv_kernel.size()

torch.Size([9, 5, 7])

In [20]:
rel_pos.size()

torch.Size([1, 1, 7])

In [19]:
ckonv.Kernel(rel_pos).size()

torch.Size([1, 45, 7])

In [27]:
import ckconv.nn.functional as ckconv_f

output = ckconv_f.causal_conv(x, conv_kernel, ckonv.bias)

In [29]:
output.size()

torch.Size([3, 9, 7])

In [21]:
def Linear1d(
    in_channels: int,
    out_channels: int,
    stride: int = 1,
    bias: bool = True,
) -> torch.nn.Module:
    """
    Implements a Linear Layer in terms of a point-wise convolution.
    """
    return nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=bias)

In [24]:
Linear1d(1, 10)(rel_pos).size()

torch.Size([1, 10, 7])